# **Task 1 objective**:

our but is to implement Retrieval-Augmented Generation (RAG) to enhance the model’s ability to generate accurate and contextually relevant responses by retrieving information from a pre-defined dataset.



**import libraries**

In [ ]:
# install necessary libraries
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers  openpyxl pacmap datasets langchain-community ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pyarrow==14.0.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.1.0
    Uninstalling pyarrow-16.1.0:
      Successfully uninstalled pyarrow-16.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.0.1 requires pyarrow>=15.0.0, but you have pyarrow 14.0.1 which is incompatible.


In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import load_dataset
import matplotlib.pyplot as plt

# Raw knowledge Base
from langchain.schema import Document as LangchainDocument
# Chunking phase :
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


# Indexation phase :
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

# Visualization :
import pacmap
import numpy as np
import plotly.express as px

# Generation phase :
from transformers import pipeline

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


# Reranking :
from ragatouille import RAGPretrainedModel

pd.set_option("display.max_colwidth", None)  # This will be helpful when visualizing retriever outputs

**load dataset**

In [ ]:
# Load the dataset
ds = load_dataset("Falah/arxiv-research-paper", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/956 [00:00<?, ?B/s]

(…)-00000-of-00007-20790aed25a8ef0e.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

(…)-00001-of-00007-36083a0c9e270f70.parquet:   0%|          | 0.00/273M [00:00<?, ?B/s]

(…)-00002-of-00007-b9ef32af27743ea2.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

(…)-00003-of-00007-b693e6170bf4a18f.parquet:   0%|          | 0.00/297M [00:00<?, ?B/s]

(…)-00004-of-00007-d72b2787def30341.parquet:   0%|          | 0.00/309M [00:00<?, ?B/s]

(…)-00005-of-00007-ba425bbdf709dee1.parquet:   0%|          | 0.00/302M [00:00<?, ?B/s]

(…)-00006-of-00007-5db8d290005bbac6.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2288300 [00:00<?, ? examples/s]

In [ ]:
ds.column_names

['id',
 'submitter',
 'authors',
 'title',
 'comments',
 'journal-ref',
 'doi',
 'report-no',
 'categories',
 'license',
 'abstract',
 'versions',
 'update_date',
 'authors_parsed']

In [ ]:
ds.features


{'id': Value(dtype='string', id=None),
 'submitter': Value(dtype='string', id=None),
 'authors': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'comments': Value(dtype='string', id=None),
 'journal-ref': Value(dtype='string', id=None),
 'doi': Value(dtype='string', id=None),
 'report-no': Value(dtype='string', id=None),
 'categories': Value(dtype='string', id=None),
 'license': Value(dtype='string', id=None),
 'abstract': Value(dtype='string', id=None),
 'versions': [{'version': Value(dtype='string', id=None),
   'created': Value(dtype='string', id=None)}],
 'update_date': Value(dtype='timestamp[s]', id=None),
 'authors_parsed': Sequence(feature=Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), length=-1, id=None)}

In [ ]:
# We take a sample

ds_ = ds.shuffle(seed=42).select(range(20000))

ds_

Dataset({
    features: ['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed'],
    num_rows: 20000
})

**A base containing documents with their richeful metadata**

In [ ]:
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=row["abstract"], metadata={"title": row["title"], "authors": row["authors"], "submitter": row["submitter"], "categories" : row["categories"],"journal reference" :row["journal-ref"]}) for row in tqdm(ds_)
]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
RAW_KNOWLEDGE_BASE[3]

Document(metadata={'title': 'Exploiting Motion Information from Unlabeled Videos for Static Image\n  Action Recognition', 'authors': 'Yiyi Zhang, Li Niu, Ziqi Pan, Meichao Luo, Jianfu Zhang, Dawei Cheng,\n  Liqing Zhang', 'submitter': 'Yiyi Zhang', 'categories': 'cs.CV', 'journal reference': 'AAAI 2020'}, page_content='  Static image action recognition, which aims to recognize action based on a\nsingle image, usually relies on expensive human labeling effort such as\nadequate labeled action images and large-scale labeled image dataset. In\ncontrast, abundant unlabeled videos can be economically obtained. Therefore,\nseveral works have explored using unlabeled videos to facilitate image action\nrecognition, which can be categorized into the following two groups: (a)\nenhance visual representations of action images with a designed proxy task on\nunlabeled videos, which falls into the scope of self-supervised learning; (b)\ngenerate auxiliary representations for action images with the gen

**Embedding the docs and chunking phase :**

In [ ]:
Markdown_seprators = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    ""
]

In [ ]:
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=Markdown_seprators,
    )

    docs_processed = []
    for doc in tqdm(knowledge_base):
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in tqdm(docs_processed):
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

# Splitting the documents


docs_processed = split_documents(
    512,  # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20456 [00:00<?, ?it/s]

Lets save the processed docs

In [ ]:
import pickle
# Save the processed documents to a file using pickle
with open('docs_processed.pkl', 'wb') as f:
    pickle.dump(docs_processed, f)

print("docs_processed has been saved as docs_processed.pkl")


docs_processed has been saved as docs_processed.pkl


**Indexation : loading the embedded chunks into the Faiss Vector DB**

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

<ipython-input-13-a731e773e1d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Save the FAISS index to disk
KNOWLEDGE_VECTOR_DATABASE.save_local('/content/drive/MyDrive/Index-faiss')


In [ ]:
# Embed a user query in the same space
user_query = "Which authors did discuss interesting theories in mathematics?"
query_vector = embedding_model.embed_query(user_query)
print(query_vector)

[-0.06432937830686569, 0.02550916187465191, 0.02887449413537979, -0.012947706505656242, -0.051122575998306274, -0.004018661566078663, -2.113063783326652e-05, 0.07091453671455383, -0.004851150792092085, -0.03779168054461479, -0.006773107685148716, -0.06125711277127266, 0.010457819327712059, 0.07959610223770142, 0.028217824175953865, -0.0004811283724848181, -0.030618969351053238, 0.04792299494147301, -0.05879389867186546, 0.025517424568533897, 0.05648724362254143, -0.043856628239154816, -0.05104554444551468, -0.06595926731824875, 0.041632603853940964, 0.03995146229863167, -0.024506906047463417, -0.05136145278811455, -0.011834652163088322, -0.17149145901203156, -0.030564194545149803, 0.010337959043681622, 0.06979945302009583, -0.028346072882413864, 0.021913936361670494, -0.024203896522521973, 0.015965266153216362, 0.06253055483102798, -0.0220020804554224, 0.04620606452226639, 0.03460071608424187, -0.029667874798178673, 0.013106669299304485, -0.032887883484363556, -0.01068185642361641, -0.

In [ ]:
# Retrieving docs related to the user's query
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print("\n==================================Top document==================================")
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)


Starting retrieval for user_query='Which authors did discuss interesting theories in mathematics?'...

==================================Top document==================================
F.: Good morning Hermann, I would like to talk with you about infinitesimals.
  G.: Tell me Pierre.
  F.: I'm fed up of all these slanders about my attitude to be non rigorous, so
I've started to study nonstandard analysis (NSA) and synthetic differential
geometry (SDG).
  G.: Yes, I've read something ...
  F.: Ok, no problem about their rigour. But, when I've seen that the sine of
an infinite in NSA is infinitely near to a real number I was astonished: what
is the intuitive meaning of this number, if any? Then, I've seen that to work
in SDG I must learn to work in intuitionistic logic ... You know, I love
margins of books, and I don't want to loose too much time, I have many things
to do ...
  G.: In SDG they also say that every infinitesimal is at the same time
positive and negative, what is the meanin

**Build Reader (Generation) Model :**

In [ ]:
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-alpha"

# Quantization :
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500
)

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Let's save the **Reader LLM**

In [ ]:
# Save the model and tokenizer
model.save_pretrained('quantized_model/')
tokenizer.save_pretrained('quantized_model/')

print("Model and tokenizer saved to 'quantized_model/' directory")

Model and tokenizer saved to 'quantized_model/' directory


**RAG_PROMPT_TEMPLATE creation**

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive detailled answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the name of the authors, do not provide the number of the document.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|system|>
Using the information contained in the context,
give a comprehensive detailled answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the name of the authors, do not provide the number of the document.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}</s>
<|assistant|>



**Constructing the context to pass to the generator LLM**

In [ ]:
retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

# First test :
final_prompt = RAG_PROMPT_TEMPLATE.format(question="Show me papers related to quantum computing with a focus on error correction!", context=context)

# Redact an answer
answer = READER_LLM(final_prompt)[0]["generated_text"]
print(answer)

The following paper discusses error correction in quantum computing:

Title: Quantum Error Correction Codes: A Tutorial
Authors: J. Preskill
Journal: Physical Review A
Volume: 75
Issue: 5
Pages: 052336
Year: 2007

This tutorial provides an overview of quantum error correction codes, which are essential for building fault-tolerant quantum computers. The paper covers topics such as the theory of quantum error correction, the encoding of logical qubits, and decoding algorithms. It also includes examples and exercises to help readers understand the concepts.

Another relevant paper is:

Title: Surface codes for fault-tolerant quantum computation
Authors: R. Raussendorf and H.-J. Briegel
Journal: Physical Review Letters
Volume: 98
Issue: 18
Pages: 187901
Year: 2007

This paper introduces surface codes, a type of topological quantum code that has been proposed as a candidate for fault-tolerant quantum computing. The paper describes the construction of the code, its properties, and how it can

**Combine all process in one function**

In [ ]:
def answer_with_rag(
    question: str,
    llm: pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [ ]:
# Rerabking the retried docs : improve the process of retrieval
RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [ ]:
question = "Show me papers related to quantum computing with a focus on error correction!"

answer, relevant_docs = answer_with_rag(question, READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER)

=> Retrieving documents...


/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00,  4.87it/s]


=> Generating answer...


In [ ]:
print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc)

==================================Answer==================================
Answer: Based on the given context, here are four papers related to quantum computing with a focus on error correction:

1. "Decohering Noise at the Logical Level" by John Preskill et al. (Document 0) presents an algorithm that decoheres noise at the logical level, projecting the state of the system onto a logical state with a well-defined error. This algorithm does not significantly increase the depth of the logical circuit and applies generally to most fault-tolerant gadgets and error correction steps.

2. "Efficient Error Mitigation Using Clifford Data Regression" by Yuan Su et al. (Document 1) introduces an error mitigation scheme that improves the frugality by carefully choosing the training data and exploiting the symmetries of the problem. This method is an order of magnitude cheaper while maintaining the same accuracy as the original Clifford data regression approach.

3. "Optimal Generator Sets for Hybr

**chatbot_response function**

In [ ]:
def chatbot_response(user_query: str) :
  retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
  retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
  context = "\nExtracted documents:\n"
  context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])
  final_prompt = RAG_PROMPT_TEMPLATE.format(question=user_query, context=context)

  # Redact an answer
  answer = READER_LLM(final_prompt)[0]["generated_text"]
  return f"Echo: {answer}"

In [ ]:
chatbot_response(user_query)

"Echo: The authors who discussed interesting theories in mathematics are not explicitly mentioned in the given context. However, based on the titles of the extracted documents, some authors who may have discussed interesting theories in mathematics include Pierre (presumably the author of Document 0), as he mentions studying nonstandard analysis (NSA) and synthetic differential geometry (SDG), and the authors of Documents 1 and 4, who reviewed the role of mathematics in data science and the significance of Emmy Noether's theorem, respectively."

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.1
    Uninstalling MarkupSafe-3.0.1:
      Successfully uninstalled MarkupSafe-3.0.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
ERROR:

In [ ]:
import gradio as gr

# Define the chatbot interface
iface = gr.Interface(
    fn=chatbot_response,       # The function to call for each user input
    inputs="text",             # The input type
    outputs="text",            # The output type
    title="Simple Echo Chatbot" # Title of the interface
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c1fbaea39a9b01df6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Lets test if **app.py** works well with model resources

In [ ]:
from flask import Flask, request, jsonify
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from ragatouille import RAGPretrainedModel
from langchain_community.vectorstores.utils import DistanceStrategy
import logging
from langchain_community.vectorstores import FAISS
import time  # Optional: for simulating time taken for indexing
import faiss
import os
from flask_ngrok import run_with_ngrok
logging.basicConfig(level=logging.DEBUG)

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)
# Initialize local paths (replace with your local paths)
LOCAL_MODEL_DIR = '/content/drive/MyDrive/quantized_model'
LOCAL_DOCS_PROCESSED = '/content/drive/MyDrive/docs_processed.pkl'


def load_resources():
    global KNOWLEDGE_VECTOR_DATABASE, embedding_model, READER_LLM, RERANKER, RAG_PROMPT_TEMPLATE
    logging.debug("Loading resources...")

    # Ensure directories for storing models exist
    os.makedirs(LOCAL_MODEL_DIR, exist_ok=True)

    # Load processed documents
    with open(LOCAL_DOCS_PROCESSED, "rb") as f:
        docs_processed = pickle.load(f)
    logging.debug("Documents processed loaded")

    # Load embedding model
    embedding_model = HuggingFaceEmbeddings(
        model_name="thenlper/gte-small",
        multi_process=True,
        model_kwargs={"device": "cuda"},  # Switch to GPU if available
        encode_kwargs={"normalize_embeddings": True}
    )
    logging.debug("Embedding model loaded")

    # Indexing docs
    total_docs = len(docs_processed)
    logging.debug("Indexing docs...")

    KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
        docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
    )


    logging.debug("Knowledge vector database initialized")

    # Load reranker model
    RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    logging.debug("Reranker model loaded")

    # Load quantized language model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL_DIR)
    tokenizer = AutoTokenizer.from_pretrained(LOCAL_MODEL_DIR)
    logging.debug("Language model and tokenizer loaded")

    # Initialize text generation pipeline
    READER_LLM = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        do_sample=True,
        temperature=0.2,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=500
    )
    logging.debug("Reader LLM initialized")

    # Define the RAG prompt template
    prompt_in_chat_format = [
        {
            "role": "system",
            "content": """Using the information contained in the context, give a comprehensive detailed answer to the question."""
        },
        {
            "role": "user",
            "content": """Context:\n{context}\n---\nNow here is the question you need to answer.\nQuestion: {question}"""
        }
    ]
    RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
        prompt_in_chat_format, tokenize=False, add_generation_prompt=True
    )
    logging.debug("RAG prompt template defined")


@app.route('/ask', methods=['POST'])
def ask():
    # Parse user query
    data = request.get_json()
    user_query = data.get("question", "")

    # Retrieve relevant documents
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
    retrieved_docs_text = [doc.page_content for doc in retrieved_docs]

    # Rerank relevant documents
    relevant_docs = RERANKER.rerank(user_query, retrieved_docs_text, k=5)
    relevant_docs = [doc["content"] for doc in relevant_docs]

    # Construct context from retrieved documents
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}::\n" + doc for i, doc in enumerate(relevant_docs)])

    # Format the final prompt
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=user_query, context=context)

    # Generate the answer from the LLM
    answer = READER_LLM(final_prompt)[0]["generated_text"]
    return jsonify({"answer": answer, "relevant_docs": relevant_docs})


# Main Flask app entry point
if __name__ == '__main__':
    load_resources()
    logging.debug("Starting Flask app...")
    app.run(host='0.0.0.0', port=5000)


In [ ]:
!pip install flask-ngrok